In [2]:
import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

# tfds works in both Eager and Graph modes
tf.enable_eager_execution()

# Load the full GMD with MIDI only (no audio) as a tf.data.Dataset
dataset = tfds.load(
    name="groove/full-midionly",
    split=tfds.Split.TRAIN,
    try_gcs=True)

# Build your input pipeline
dataset = dataset.shuffle(1024).batch(32).prefetch(
    tf.data.experimental.AUTOTUNE)
for features in dataset.take(1):
  # Access the features you are interested in
  midi, genre = features["midi"], features["style"]["primary"]


In [3]:
print(dataset)

<PrefetchDataset element_spec={'bpm': TensorSpec(shape=(None,), dtype=tf.int32, name=None), 'drummer': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'id': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'midi': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'style': {'primary': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'secondary': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, 'time_signature': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'type': TensorSpec(shape=(None,), dtype=tf.int64, name=None)}>


In [ ]:
for features in dataset.take(1):
  # Access the features you are interested in
  midi, genre = features["midi"], features["style"]["primary"]
  print(midi, genre)

## 과제 상세안내

- 해당 과제는 `MusicVAE`를 구현하는 것입니다.

그 중에서도 Groove MIDI Dataset을 이용하여 4마디에 해당하는 드럼 샘플을 뽑아내는 과정을 수행하시면 됩니다.

- 깃헙 코드: https://github.com/magenta/magenta/tree/master/magenta/models/music_vae

- 관련 논문: https://arxiv.org/pdf/1803.05428.pdf

- 관련 데이터: https://magenta.tensorflow.org/datasets/groove

미디라는 데이터는 음악이 어떻게 연주되어야 하는지 나타내는 악보와 비슷한 개념으로 이해하시면 됩니다. 어느 시점에 어떤 악기를 연주하여야 하는지가 나와있는 데이터이며 .midi의 확장자명을 가지고 있습니다.

설명드린 코드에서는 해당 미디를 학습에 이용하기 위하여 벡터로 변환하는 전처리 과정이 필요합니다.

이러한 전처리 과정을 거치면 tfrecord형식으로 저장되게 되며 이것을 학습하는 형태입니다.

- 과제는 어떠한 방식으로 모델을 다루는지를 보기 위함이며, 과제를 100% 완수하지 못하셔도 됩니다.

# 사전 지식 탐색 및 정리

## VAE
- VAE는 latent representation에 효과적인 모델로 입증되었음
- AE와 형태면에서 비슷함. Bottleneck 구조를 통해 벡터를 압축하여 latent code를 생성. 
  - latent space를 통해 입력 데이터간의 유사성과 차이를 통해 학습
  - VAE는 AE와는 달리 Generative한 모델임.
  - 생성에 강점을 보이는 이유는 latent space가 연속되게 생성되기 때문. 반면 AE는 이산적인 분포를 가지기 때문에 데이터 재구성 단계에서 저품질의 데이터가 생기는 원인이 됨.
  - VAE는 KL-divergence를 loss 함수로 사용함
- KL-divergence를 통해서 VAE의 latent space 분포가 `표준 정규 분포`에 가깝도록 학습하게 됨

>참고링크: https://ratsgo.github.io/generative%20model/2017/12/19/vi/


---


## VAE 한계점

- sequential data에 한계점이 존재함.
- recurrent VAE 모델들은 long-term sequence에 약함
- 논문에서는 이러한 한계점을 극복하기 위해 `hierachical decoder`를 제안함
  - 첫번째 output embedding이 input의 sub-sequence로 쓰임
  - 각 embedding들은 독립적으로 sub-sequence들을 생성함
- 이러한 구조는 모델이 latent code를 utilize하고 `posterior collapse` 문제를 회피하는데 도움이 됨

  - posterior collapse: 시퀀스 모델을 VAE형태로 표현하면 global latent z를 이용하여 다양한 속성의 시퀀스를 생성할 수 있다. 하지만 이 때 decoder가 encoder의 condition을 무시하고 시퀀스를 생성하는 현상을 말함.

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fqd1lx%2FbtqXMfe0scF%2FYRdkqrZhcInlZ2yPRchOzK%2Fimg.png)

  - collapse 발생하는 이유
    - decoder가 latent z 없이 과거 데이터만으로 충분히 생성 가능한 경우
    - 다양한 latent z가 존재할 수 있음
    - VAE가 local information을 선호하는 경향이 있음
    - 학습 초기 encoder가 z를 잘 표현하지 못 해서 등등..
    - 참고링크: https://stopspoon.tistory.com/63

# Model
- RNN은 recurrent한 모델이기 때문에 latent code를 무시할 수 있음
- 전체 시퀀스를 단일 latent vector로 압축하게 되는데 따라서 long-term 시퀀스인 경우 정보의 손실일 일어날 수 있음
- 이 같은 문제를 해결하기 위해 `hierarchical RNN`을 디코더에 적용

![](https://i.imgur.com/y397wfb.png)

## Bidirectional Encoder
- 2 layer 양방향 LSTM 사용
- input 시퀀스 x로 첫번째 state vector $ \overrightarrow{h_{T}} $ 와 $ \overleftarrow{h_{T}} $ 를 두번째 bi-direction LSTM layer에서 얻는다.
- 이는 concat되어 $h_{T}$를 얻고 2개의 fully connectd layer로 들어간다.

$$ \mu = W_{h\mu}h_{T} + b_{\mu} $$

$$ σ = log(exp(W_{hσ} + b_{σ}) + 1) $$

$W_{hμ}, W_{hσ}$는 가중치 행렬이고, $b_μ, b_σ$는 bias 벡터이다. 논문에서는 2048개의 layer와 512의 latent 차원을 사용한다. 일반적인 VAE처럼 뮤와 시그마를 통해서 latent 분포를 표현할 수 있다. 양방향 recurrent encoder는 이상적으로 input 시퀀스의 long-term context를 잘 표현할 수 있다고 함.

## Hierarchical Decoder
- 단층 RNN을 사용한 경우 -> 매우 떨어지는 샘플링 및 재구성 성능을 보임
  - 출력 시퀀스가 생성되면서 latent state의 영향력이 점차 소실되기 때문 (vanishing)
- 이러한 문제를 해결하기 위해 계층적 RNN 구조를 디코더에 적용함
- input 시퀀스 x를 중복되지 않는 U개의 `sub sequence` $y_u$로 나누게 되면 다음처럼 표현 할 수 있음. 여기서 $i_u$는 endpoint임

$$ y_u = \{ x_{i_u}, x_{i_u+1}, x_{i_u+2}, ..., x_{i_u+1} -1 \} $$

$$ →X = \{ y_1, y_2, ..., y_U \} $$

- 위 식에서 $i_{U+1} = T$로 정의함. 이후 latent vector z는 fully-connetced layer를 통과하며 tanh 활성 함수를 거쳐 conductor RNN의 초기 state를 생성함.

- 논문에서는 Conductor를 위해서 2개층의 양방향 LSTM (hidden state size 1024, 512 output dimension)을 사용함.

### Conductor

- conductor가 임베딩 벡터 c를 생성하면 각 벡터는 독립적으로 fully-connected layer를 지나면서 tanh 활성함수를 통해 RNN decoder의 최종 바닥층의 초기 state를 생성함.
- RNN 디코더는 softmax 출력층을 거치면서 auto-regressive하게 시퀀스 분포를 생성함. 여기서 디코더는 서브시퀀스 $y_{u}$에 해당하는 분포를 생성함.
- 디코더의 바닥층에서는 매 step마다 conductor의 임베딩 $c_u$와 이전 step의 출력 토큰을 결합하여 input으로 사용함.
  - 이러한 계층적 구조때문에 각 서브시퀀스 $y_u$는 연결된 conductor에서 생성된 $c_u$를 통해서만 영향을 받음
- 논문에서는 2 layer LSTM과 각 layer마다 1024 units의 decoder RNN을 사용함.

### 결과

- 논문에서는 실험을 통해 decoder의 scope를 제한하는 것이 latent code를 사용하는 long-term 구조 모델에 중요하다는 사실을 발견함
- 각 서브시퀀스의 끝에서 decoder의 state가 다시 conductor로 들어가는 `auto regressive conductor`에서는 성능이 좋지 않았음.





# Experiments

## Data and Training

MIDI 데이터를 학습에 사용하기 위해서 벡터로 변환하는 전처리 과정이 필요하다.

[magenta](https://github.com/magenta/magenta/tree/main/magenta/models/music_vae) 를 이용하여 .mid, .midi 파일을 tfrecord 포멧으로 변환시킬 수 있다. 이후 학습에 사용하면 된다!

* tfrecord: 바이너리 데이터 포멧으로 serial 데이터를 읽는데 특화되어 있다. [참고링크](https://bcho.tistory.com/1190)

In [ ]:
!pip install magenta

In [37]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive/')

# data dir
DATA_DIR = '/content/drive/MyDrive/musicVAE/data/groove/drummer1'
OUT_DIR = '/content/drive/MyDrive/musicVAE/data_out/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!curl https://raw.githubusercontent.com/tensorflow/magenta/main/magenta/tools/magenta-install.sh > /tmp/magenta-install.sh
!bash /tmp/magenta-install.sh

In [32]:
from magenta.scripts import convert_dir_to_note_sequences as mg
import tensorflow as tf
import numpy as np
# convert_dir_to_note_sequences

In [39]:
#writer = tf.data.experimental.TFRecordWriter('test.tfrecord')
writer = tf.io.TFRecordWriter('test.tfrecord')

mg.convert_files(DATA_DIR, OUT_DIR, writer=writer)


INFO:tensorflow:Converting files in '/content/drive/MyDrive/musicVAE/data_out/'.
